In [45]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("nehalbirla/vehicle-dataset-from-cardekho")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\rohit\.cache\kagglehub\datasets\nehalbirla\vehicle-dataset-from-cardekho\versions\4


In [46]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score

In [47]:
car_dataset = pd.read_csv(r"C:\Users\rohit\OneDrive\Desktop\mach_learning\Car_Price_Prediction\Car details v3.csv")

In [48]:
car_dataset.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.4 kmpl,1248 CC,74 bhp,190Nm@ 2000rpm,5.0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14 kmpl,1498 CC,103.52 bhp,250Nm@ 1500-2500rpm,5.0
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,17.7 kmpl,1497 CC,78 bhp,"12.7@ 2,700(kgm@ rpm)",5.0
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,23.0 kmpl,1396 CC,90 bhp,22.4 kgm at 1750-2750rpm,5.0
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,16.1 kmpl,1298 CC,88.2 bhp,"11.5@ 4,500(kgm@ rpm)",5.0


In [49]:
car_dataset.isnull().sum()

name               0
year               0
selling_price      0
km_driven          0
fuel               0
seller_type        0
transmission       0
owner              0
mileage          221
engine           221
max_power        215
torque           222
seats            221
dtype: int64

In [50]:
car_dataset["mileage"] = car_dataset["mileage"].str.extract(r'(\d+\.?\d*)')
car_dataset["mileage"] = car_dataset["mileage"].astype(float)
car_dataset["mileage"].fillna(car_dataset["mileage"].median(), inplace=True)

In [51]:
car_dataset["engine"] = car_dataset["engine"].str.extract(r'(\d+\.?\d*)')
car_dataset["engine"] = car_dataset["engine"].astype(float)
car_dataset["engine"].fillna(car_dataset["engine"].median(), inplace=True)

In [52]:
car_dataset["max_power"] = car_dataset["max_power"].str.extract(r'(\d+\.?\d*)')
car_dataset["max_power"] = car_dataset["max_power"].astype(float)
car_dataset["max_power"].fillna(car_dataset["max_power"].median(), inplace=True)

In [53]:
car_dataset["torque"] = car_dataset["torque"].str.extract(r'(\d+\.?\d*)')
car_dataset["torque"] = car_dataset["torque"].astype(float)
car_dataset["torque"].fillna(car_dataset["torque"].median(), inplace=True)

In [54]:
car_dataset["seats"].fillna(car_dataset["seats"].mode()[0], inplace=True)

In [55]:
car_dataset.isnull().sum()

name             0
year             0
selling_price    0
km_driven        0
fuel             0
seller_type      0
transmission     0
owner            0
mileage          0
engine           0
max_power        0
torque           0
seats            0
dtype: int64

In [56]:
car_dataset.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.40,1248.0,74.00,190.0,5.0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14,1498.0,103.52,250.0,5.0
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,17.70,1497.0,78.00,12.7,5.0
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,23.00,1396.0,90.00,22.4,5.0
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,16.10,1298.0,88.20,11.5,5.0


In [57]:
car_dataset.value_counts()

name                                   year  selling_price  km_driven  fuel    seller_type       transmission  owner        mileage  engine  max_power  torque  seats
Lexus ES 300h                          2019  5150000        20000      Petrol  Dealer            Automatic     First Owner  22.37    2487.0  214.56     202.0   5.0      34
Jaguar XF 2.0 Diesel Portfolio         2017  3200000        45000      Diesel  Dealer            Automatic     First Owner  19.33    1999.0  177.00     430.0   5.0      34
Toyota Camry 2.5 Hybrid                2016  2000000        68089      Petrol  Trustmark Dealer  Automatic     First Owner  19.16    2494.0  157.70     213.0   5.0      32
Honda Jazz VX                          2016  550000         56494      Petrol  Trustmark Dealer  Manual        First Owner  18.20    1199.0  88.70      110.0   5.0      32
Honda Amaze V CVT Petrol BSIV          2019  779000         7032       Petrol  Trustmark Dealer  Automatic     First Owner  19.00    1199.0  88.76

In [58]:
car_dataset['fuel'].value_counts()

fuel
Diesel    4402
Petrol    3631
CNG         57
LPG         38
Name: count, dtype: int64

In [59]:
car_dataset['seller_type'].value_counts()

seller_type
Individual          6766
Dealer              1126
Trustmark Dealer     236
Name: count, dtype: int64

In [60]:
car_dataset['transmission'].value_counts()

transmission
Manual       7078
Automatic    1050
Name: count, dtype: int64

In [61]:
car_dataset['owner'].value_counts()

owner
First Owner             5289
Second Owner            2105
Third Owner              555
Fourth & Above Owner     174
Test Drive Car             5
Name: count, dtype: int64

In [62]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

# Encode categorical columns
car_dataset['fuel_encoded'] = le.fit_transform(car_dataset['fuel'])
car_dataset['seller_type_encoded'] = le.fit_transform(car_dataset['seller_type'])
car_dataset['transmission_encoded'] = le.fit_transform(car_dataset['transmission'])
car_dataset['owner_encoded'] = le.fit_transform(car_dataset['owner'])

# Show the encoded dataset
car_dataset[['fuel', 'fuel_encoded', 'seller_type', 'seller_type_encoded', 'transmission', 'transmission_encoded', 'owner', 'owner_encoded']].head(10)

,fuel,fuel_encoded,seller_type,seller_type_encoded,transmission,transmission_encoded,owner,owner_encoded
0,Diesel,1,Individual,1,Manual,1,First Owner,0
1,Diesel,1,Individual,1,Manual,1,Second Owner,2
2,Petrol,3,Individual,1,Manual,1,Third Owner,4
3,Diesel,1,Individual,1,Manual,1,First Owner,0
4,Petrol,3,Individual,1,Manual,1,First Owner,0
5,Petrol,3,Individual,1,Manual,1,First Owner,0
6,LPG,2,Individual,1,Manual,1,First Owner,0
7,Petrol,3,Individual,1,Manual,1,Second Owner,2
8,Diesel,1,Individual,1,Manual,1,First Owner,0
9,Diesel,1,Individual,1,Manual,1,First Owner,0


In [63]:
# Check what columns exist
print("Current columns:", car_dataset.columns.tolist())
print("\nDataset shape:", car_dataset.shape)
car_dataset.head()

Current columns: ['name', 'year', 'selling_price', 'km_driven', 'fuel', 'seller_type', 'transmission', 'owner', 'mileage', 'engine', 'max_power', 'torque', 'seats', 'fuel_encoded', 'seller_type_encoded', 'transmission_encoded', 'owner_encoded']

Dataset shape: (8128, 17)


,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats,fuel_encoded,seller_type_encoded,transmission_encoded,owner_encoded
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.40,1248.0,74.00,190.0,5.0,1,1,1,0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14,1498.0,103.52,250.0,5.0,1,1,1,2
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,17.70,1497.0,78.00,12.7,5.0,3,1,1,4
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,23.00,1396.0,90.00,22.4,5.0,1,1,1,0
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,16.10,1298.0,88.20,11.5,5.0,3,1,1,0


In [64]:
print(car_dataset.columns)

Index(['name', 'year', 'selling_price', 'km_driven', 'fuel', 'seller_type',
       'transmission', 'owner', 'mileage', 'engine', 'max_power', 'torque',
       'seats', 'fuel_encoded', 'seller_type_encoded', 'transmission_encoded',
       'owner_encoded'],
      dtype='object')


In [65]:
car_dataset.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats,fuel_encoded,seller_type_encoded,transmission_encoded,owner_encoded
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.40,1248.0,74.00,190.0,5.0,1,1,1,0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14,1498.0,103.52,250.0,5.0,1,1,1,2
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,17.70,1497.0,78.00,12.7,5.0,3,1,1,4
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,23.00,1396.0,90.00,22.4,5.0,1,1,1,0
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,16.10,1298.0,88.20,11.5,5.0,3,1,1,0


In [77]:
# First, drop the original categorical columns if they still exist
cols_to_drop = ["selling_price", "name", "fuel", "seller_type", "transmission", "owner"]
cols_to_drop = [col for col in cols_to_drop if col in car_dataset.columns]

X = car_dataset.drop(cols_to_drop, axis=1)
Y = car_dataset["selling_price"]

print("Features (X) shape:", X.shape)
print("Features (X) columns:", X.columns.tolist())
print("\nTarget (Y) shape:", Y.shape)
print("\nFirst few rows of X:")
print(X.head())
print("\nFirst few rows of Y:")
print(Y.head())

Features (X) shape: (8128, 11)
Features (X) columns: ['year', 'km_driven', 'mileage', 'engine', 'max_power', 'torque', 'seats', 'fuel_encoded', 'seller_type_encoded', 'transmission_encoded', 'owner_encoded']

Target (Y) shape: (8128,)

First few rows of X:
   year  km_driven  ...  transmission_encoded  owner_encoded
0  2014     145500  ...                     1              0
1  2014     120000  ...                     1              2
2  2006     140000  ...                     1              4
3  2010     127000  ...                     1              0
4  2007     120000  ...                     1              0

[5 rows x 11 columns]

First few rows of Y:
0    450000
1    370000
2    158000
3    225000
4    130000
Name: selling_price, dtype: int64


In [78]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

In [79]:
model = RandomForestRegressor()
model.fit(X_train, Y_train)

,n_estimators,100
,criterion,'squared_error'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [84]:
training_data_prediction = model.predict(X_train)
training_data_accuracy = model.score(X_train, Y_train)

print("Training Data Accuracy:", training_data_accuracy)

Training Data Accuracy: 0.9956653799984028


In [85]:
test_data_prediction = model.predict(X_test)
test_data_accuracy = model.score(X_test, Y_test)
print("Test Data Accuracy:", test_data_accuracy)

Test Data Accuracy: 0.9645696973018135
